# Business Question
## How do rising condo costs, as measured by rental prices and condo values, influence homelessness rates in San Jose, California?

Why California?
- Homless people are not looking to rent out multifamily/single family homes
- condos are the most affordable for homless people
- The recent wildfires might have an affect on the future rental prices of condos
- California is one of the leading states within the US that has the most homeless population

# Libraries Needed To Run

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

# Cleaning/Familiarize with dataset

1. Load the condo price data and understand the different variables
2. Replace missing values (took a long time to figure out what I wanted to do. Do I want to delete the rows with missing values? Do i delete columns that have more than 50% values missing?
3. Load the condo rental price data and understand the different variables

### Condo Price Data Wrangling

In [ ]:
# Load the uploaded dataset to inspect its structure and contents
file_path = '/Users/student/Documents/Spring Babson 2025/Advanced Programming/Week 1 Review Pandas/Homeless Assignment/data/Orignial Data/CondoPriceMetro.csv'

data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head(), data.info()

In [ ]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Display columns with missing values and their count
print("Missing values per column:")
print(missing_values[missing_values > 0])


In [ ]:
# This finds the average percentage change for each row (not including any missing values) then fills in the missing datapoints going 
# backwards subtracting the current price/1+%change to find the previous answer. It skips a datapoint if it already has a number in it

# Ensure all price-related columns are numeric
price_columns = data.columns[5:]  # Exclude the first 5 columns
data[price_columns] = data[price_columns].apply(pd.to_numeric, errors='coerce')

# Function to fill missing values using the average percentage change
def fill_missing_values_with_avg_percentage_change(data, price_columns):
    # Calculate the average percentage change for each row
    row_percentage_changes = data[price_columns].pct_change(axis=1)
    average_percentage_changes = row_percentage_changes.mean(axis=1, skipna=True)

    # Fill missing values working backward from the most recent column
    for col in reversed(price_columns):  # Iterate through price columns in reverse order
        for i in data.index:  # Iterate over each row
            if pd.isnull(data.loc[i, col]):  # Check if the value is missing
                # Find the next non-missing value in the same row
                next_col = price_columns[price_columns.get_loc(col) + 1] if price_columns.get_loc(col) + 1 < len(price_columns) else None
                if next_col and not pd.isnull(data.loc[i, next_col]):  # Ensure the next column exists and has a value
                    next_value = data.loc[i, next_col]
                    avg_change = average_percentage_changes[i]
                    
                    # Fill the missing value using the average percentage change
                    if pd.notnull(avg_change):
                        data.loc[i, col] = next_value / (1 + avg_change)  # Back-calculate the value
    return data

# Apply the function to fill missing values
data_filled = fill_missing_values_with_avg_percentage_change(data, price_columns)

# Check if there are still missing values
missing_values_after_filling = data_filled[price_columns].isnull().sum()

# Display the result
print("Missing values after filling:")
print(missing_values_after_filling[missing_values_after_filling > 0])


In [ ]:
# Check for missing values in each column
missing_values = data_filled.isnull().sum()

# Display columns with missing values and their count
print("Missing values per column:")
print(missing_values[missing_values > 0])

In [ ]:
# Calculate the average prices across regions
avg_prices = data_filled[price_columns].mean(axis=1)

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(avg_prices, bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Average Prices Across Regions', fontsize=14)
plt.xlabel('Average Price', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


### Condo Rental Price Data Wrangling

In [ ]:
# Define the file path (adjust if the file is in a different location)
file_path_2 = '/Users/student/Documents/Spring Babson 2025/Advanced Programming/Week 1 Review Pandas/Homeless Assignment/data/Orignial Data/CondoRentalMetro.csv'

# Load the dataset
condo_data = pd.read_csv(file_path_2)

# Display the first few rows of the dataset
condo_data.head()


In [ ]:
# Check for missing data in each column
missing_data = condo_data.isnull().sum()

# Display columns with missing data and the count of missing values
missing_data = missing_data[missing_data > 0]
print("Columns with missing data and their counts:")
print(missing_data)

In [ ]:
# Identify the columns corresponding to date variables
date_columns = condo_data.columns[5:]  # Assuming the first 5 columns are not date variables

# Remove rows with missing data in any of the date columns
condo_data_cleaned = condo_data.dropna(subset=date_columns)

# Check the number of rows remaining
remaining_rows = condo_data_cleaned.shape[0]

print(f"Number of rows remaining after removing rows with missing data in date columns: {remaining_rows}")

# Display the first few rows of the cleaned dataset
condo_data_cleaned.head()


In [ ]:
# Check for missing data in each column
missing_data = condo_data_cleaned.isnull().sum()

# Display columns with missing data and the count of missing values
missing_data = missing_data[missing_data > 0]
print("Columns with missing data and their counts:")
print(missing_data)

In [ ]:
# Calculate the average rental price for each row in the cleaned dataset, excluding missing values
average_rental_prices_cleaned = condo_data_cleaned[date_columns].mean(axis=1, skipna=True)

# Create a histogram to visualize the distribution of average rental prices for the cleaned dataset
plt.figure(figsize=(10, 6))
plt.hist(average_rental_prices_cleaned, bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Average Rental Prices', fontsize=16)
plt.xlabel('Average Rental Price', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


# Data Analysis

1. Analyse the trend overtime of condo prices in San Jose, California
2. Analyse the trend overtime of cost of renting a condo in San Jose, Califronia
3. 1 Visualization for each price trend and rental price trend

### 1. Analyse condo price trends for San Jose compared to rest of united states

In [ ]:
# Filter data for San Jose and United States
filtered_data = data_filled[data_filled['RegionName'].isin(['San Jose, CA', 'United States'])]

# Calculate statistics for each region
statistics = {}
for region in filtered_data['RegionName'].unique():
    region_data = filtered_data[filtered_data['RegionName'] == region][price_columns]
    
    # Average price change (percentage change row-wise)
    avg_price_change = region_data.pct_change(axis=1).mean(axis=1, skipna=True).mean()
    
    # Maximum price
    max_price = region_data.max().max()
    
    # Minimum price
    min_price = region_data.min().min()
    
    # Mean price
    mean_price = region_data.mean().mean()
    
    # Store results
    statistics[region] = {
        'Average Price Change (%)': avg_price_change * 100,  # Convert to percentage
        'Max Price ($)': max_price,
        'Min Price ($)': min_price,
        'Mean Price ($)': mean_price
    }

# Prepare data for visualization
regions = list(statistics.keys())
max_prices = [stats['Max Price ($)'] for stats in statistics.values()]
min_prices = [stats['Min Price ($)'] for stats in statistics.values()]
mean_prices = [stats['Mean Price ($)'] for stats in statistics.values()]

# Create a bar chart
fig, ax1 = plt.subplots(figsize=(12, 6))

# Bar width
bar_width = 0.25

# Bar positions
bar_positions = range(len(regions))
bar_positions_max = [pos for pos in bar_positions]
bar_positions_min = [pos + bar_width for pos in bar_positions]
bar_positions_mean = [pos + 2 * bar_width for pos in bar_positions]

# Plot data
ax1.bar(bar_positions_max, max_prices, bar_width, label='Max Price ($)', color='green', alpha=0.7)
ax1.bar(bar_positions_min, min_prices, bar_width, label='Min Price ($)', color='red', alpha=0.7)
ax1.bar(bar_positions_mean, mean_prices, bar_width, label='Mean Price ($)', color='blue', alpha=0.7)

# Add labels, title, and legend
ax1.set_title('Statistics for San Jose and United States', fontsize=14)
ax1.set_xticks([pos + bar_width for pos in bar_positions])
ax1.set_xticklabels(regions, fontsize=12)
ax1.set_ylabel('Value ($)', fontsize=12)
ax1.legend()

# Display grid and adjust layout
ax1.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Display the results with formatting
for region, stats in statistics.items():
    print(f"Statistics for {region}:")
    print(f"  - Average Price Change: {stats['Average Price Change (%)']:.2%}")
    print(f"  - Max Price: ${stats['Max Price ($)']:,.2f}")
    print(f"  - Min Price: ${stats['Min Price ($)']:,.2f}")
    print(f"  - Mean Price: ${stats['Mean Price ($)']:,.2f}")
    print()



While this analysis focuses on condo prices in San Jose, CA, the observed high mean and maximum prices, combined with a steady rate of price increases, suggest that rising housing costs could exacerbate financial strain on residents. These elevated and consistently increasing condo prices reflect a market that may be out of reach for many, potentially contributing to housing instability. If homelessness rates are correlated with affordability challenges, the rising costs in San Jose are likely a key driver. 

### 2. Analysis Of Rental Price Trends in San Jose compared to United States

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Filter data for San Jose and United States
condo_filtered_data = condo_data_cleaned[condo_data_cleaned['RegionName'].isin(['San Jose, CA', 'United States'])]

# Exclude prices greater than 130 for San Jose
condo_filtered_data.loc[condo_filtered_data['RegionName'] == 'San Jose, CA', date_columns] = condo_filtered_data.loc[
    condo_filtered_data['RegionName'] == 'San Jose, CA', date_columns
].applymap(lambda x: x if x <= 130 else np.nan)

# Calculate statistics for each region
statistics = {}
for region in condo_filtered_data['RegionName'].unique():
    region_data = condo_filtered_data[condo_filtered_data['RegionName'] == region][date_columns]
    
    # Average price change (percentage change row-wise)
    avg_price_change = region_data.pct_change(axis=1).mean(axis=1, skipna=True).mean()
    
    # Maximum price
    max_price = region_data.max().max()
    
    # Minimum price
    min_price = region_data.min().min()
    
    # Mean price
    mean_price = region_data.mean().mean()
    
    # Store results
    statistics[region] = {
        'Average Price Change (%)': avg_price_change * 100,  # Convert to percentage
        'Max Price ($)': max_price,
        'Min Price ($)': min_price,
        'Mean Price ($)': mean_price
    }

# Prepare data for visualization
regions = list(statistics.keys())
max_prices = [stats['Max Price ($)'] for stats in statistics.values()]
min_prices = [stats['Min Price ($)'] for stats in statistics.values()]
mean_prices = [stats['Mean Price ($)'] for stats in statistics.values()]

# Create a bar chart
fig, ax1 = plt.subplots(figsize=(12, 6))

# Bar width
bar_width = 0.25

# Bar positions
bar_positions = range(len(regions))
bar_positions_max = [pos for pos in bar_positions]
bar_positions_min = [pos + bar_width for pos in bar_positions]
bar_positions_mean = [pos + 2 * bar_width for pos in bar_positions]

# Plot data
ax1.bar(bar_positions_max, max_prices, bar_width, label='Max Price ($)', color='green', alpha=0.7)
ax1.bar(bar_positions_min, min_prices, bar_width, label='Min Price ($)', color='red', alpha=0.7)
ax1.bar(bar_positions_mean, mean_prices, bar_width, label='Mean Price ($)', color='blue', alpha=0.7)

# Add labels, title, and legend
ax1.set_title('Statistics for San Jose and United States (San Jose Prices ≤ $130)', fontsize=14)
ax1.set_xticks([pos + bar_width for pos in bar_positions])
ax1.set_xticklabels(regions, fontsize=12)
ax1.set_ylabel('Value ($)', fontsize=12)
ax1.legend()

# Display grid and adjust layout
ax1.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Display the results with formatting
for region, stats in statistics.items():
    print(f"Statistics for {region}:")
    print(f"  - Average Price Change: {stats['Average Price Change (%)']:.2f}%")
    print(f"  - Max Price: ${stats['Max Price ($)']:,.2f}")
    print(f"  - Min Price: ${stats['Min Price ($)']:,.2f}")
    print(f"  - Mean Price: ${stats['Mean Price ($)']:,.2f}")
    print()


Based on the anlaysis of the rental prices, San Jose's average condo rental prices are similar to the rest of the united states after filtering the cost below 130 dolalrs. I filtered the costs below 130 becuase if you look at the trend line for rental prices in San Jose there is a massive jump from 2021 to 2022. This is probably caused by COVID and since then rental prices have dropped and have not gone up above 130. If you look at United States trend line there is an uptik in 2021 but it slowly declines as the years progress making it not as much of an outliar. If I were to include every price in my comparison, the data would be skewed and we would have some reason to beleive that the rises in rental prices could cause homelesness as people cannot afford the prices. There is some limitations to this dataset, whihc is that we only have 4 years of data which I dont beleive is enough to look at the relationship between rising rental prices and homelessness in San Jose.

### Visualizations

In [ ]:
# Filter data for the United States
us_data = condo_data_cleaned[condo_data_cleaned['RegionName'] == 'United States']

# Extract date columns and ensure they are numeric
date_columns = condo_data_cleaned.columns[5:]  # Assuming the first 5 columns are metadata
us_prices = us_data[date_columns].mean()  # Use the mean if there are multiple rows

# Plot the trend line for United States rental prices
plt.figure(figsize=(12, 6))
plt.plot(date_columns, us_prices, label='United States Rental Prices', marker='o', color='blue')
plt.title('United States Rental Prices Over Time', fontsize=16)
plt.xlabel('Time', fontsize=14)
plt.ylabel('Rental Price', fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:


# Define the exponential model
def exponential_model(x, a, b, c):
    return a * np.exp(b * x) + c

# Filter data for San Jose
san_jose_data = condo_data_cleaned[condo_data_cleaned['RegionName'].str.contains("San Jose", case=False, na=False)]

# Extract date columns and ensure they are numeric
date_columns = condo_data_cleaned.columns[5:]  # Assuming the first 5 columns are metadata
san_jose_prices = san_jose_data[date_columns].mean()  # Calculate mean if multiple rows exist

# Prepare the historical data
time_indices = np.arange(len(date_columns))  # Historical time indices
prices = san_jose_prices.values  # Historical prices

# Fit the exponential model to the historical data
popt, _ = curve_fit(exponential_model, time_indices, prices, p0=(1, 0.01, 1))

# Extend the time indices for prediction (next 2 years)
future_indices = np.arange(len(date_columns) + 24)

# Use the fitted model to predict prices
future_predictions = exponential_model(future_indices, *popt)

# Combine historical and predicted data for visualization
all_prices = np.concatenate((prices, future_predictions[len(date_columns):]))

# Create labels for future dates
future_dates = [f"Future-{i+1}" for i in range(24)]
all_labels = list(date_columns) + future_dates

# Plot the actual trend and exponential prediction
plt.figure(figsize=(12, 6))
plt.plot(all_labels[:len(date_columns)], prices, label='Actual Prices', marker='o', color='blue')
plt.plot(all_labels, all_prices, label='Exponential Model Prediction', linestyle='--', color='green')
plt.axvline(x=len(date_columns) - 1, color='red', linestyle='--', label='Prediction Start')
plt.title('San Jose Rental Prices with Exponential Model Predictions', fontsize=16)
plt.xlabel('Time', fontsize=14)
plt.ylabel('Rental Price', fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:

# Extract price columns (assuming they start from the 6th column)
price_columns = data_filled.columns[5:]

# Filter data for San Jose
san_jose_data = data_filled[data_filled['RegionName'] == 'San Jose, CA']

# Extract price columns and calculate the average price trend for San Jose
san_jose_prices = san_jose_data[price_columns].mean()

# Prepare data for linear regression
# Convert the date columns into numeric indices (e.g., 0, 1, 2, ..., N)
time_indices = np.arange(len(price_columns)).reshape(-1, 1)  # Reshape for sklearn
prices = san_jose_prices.values  # Convert to array

# Fit a linear regression model
model = LinearRegression()
model.fit(time_indices, prices)

# Predict future prices for 12 additional months
future_indices = np.arange(len(price_columns), len(price_columns) + 12).reshape(-1, 1)
future_predictions = model.predict(future_indices)

# Combine historical and predicted prices
extended_prices = np.concatenate([prices, future_predictions])
extended_dates = np.concatenate([price_columns, [f"Predicted {i+1}" for i in range(12)]])

# Plot the trend line with predictions
plt.figure(figsize=(14, 7))
plt.plot(range(len(prices)), prices, marker='o', linestyle='-', color='blue', alpha=0.7, label='Historical Prices')
plt.plot(range(len(prices), len(extended_prices)), future_predictions, marker='o', linestyle='--', color='orange', label='Predicted Prices')
plt.title('Condo Prices Trend and Prediction in San Jose, CA', fontsize=16)
plt.xlabel('Time (Years)', fontsize=12)
plt.ylabel('Average Condo Price ($)', fontsize=12)
plt.xticks(ticks=np.arange(0, len(extended_dates), 12), labels=[d.split('/')[2] if '/' in d else d for d in extended_dates[::12]], rotation=45)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


I tried my best with different libraries and working with chatGPT to make the prediciton line connecting, but from my analysis I think it is meant to be connecting its just the graph is so large it cant go any further. 

In [ ]:

# Extract price columns (assuming they start from the 6th column)
price_columns = data_filled.columns[5:]

# Filter data for United States
us_data = data_filled[data_filled['RegionName'] == 'United States']

# Extract price columns and calculate the average price trend for the United States
us_prices = us_data[price_columns].mean()

# Extract years from the price columns for x-axis labels
years = [col.split('/')[2] if '/' in col else col for col in price_columns]

# Create a trend line plot
plt.figure(figsize=(14, 7))
plt.plot(price_columns, us_prices, marker='o', linestyle='-', color='blue', alpha=0.7)
plt.title('Condo Prices Trend in the United States', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Average Condo Price ($)', fontsize=12)
plt.xticks(ticks=range(0, len(price_columns), 12), labels=years[::12], rotation=45)  # Show one label per year
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Conclusion

Rising condo prices in San Jose, evidenced by consistently high and increasing mean, maximum, and minimum prices compared to national averages in both condo prices and rental prices, highlight the city’s severe affordability challenges. This sustained growth in housing costs outpaces the economic capacity of many residents, pushing low- and middle-income households toward housing insecurity as they cannot meet the increasing prices of condos. Predictions indicate that prices will continue to rise, further increasing the affordability crisis and increasing homelessness rates. Addressing this issue requires targeted policies such as affordable housing development, rent stabilization, and supportive housing initiatives to counteract the pressures of escalating housing costs. In contrast, additional analysis needs to be done as there are other factors affect homelesness rates around the United States like income, but rising prices are definatley a factor. 
